# NYC Facades Condition Alart

workflow:
1. load csv
2. separate 'Safe' and 'Unsafe' building
3. for each group, scrape a photo of the facade according to the building's address
4. split training and testing data
5. build model to predict a if a facade is safe or not

Import libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests as r

## Extract

In [3]:
url_facade_data = "https://data.cityofnewyork.us/resource/xubg-57si.csv"

In [4]:
facade_con = pd.read_csv(url_facade_data)

## Examin

In [5]:
# lift column disply limit to see more columns
pd.set_option('display.max_columns', None)

In [6]:
#preview data
facade_con.head()

,tr6_no,control_no,filing_type,cycle,bin,house_no,street_name,borough,block,lot,sequence_no,submitted_on,current_status,qewi_name,qewi_bus_name,qewi_bus_street_name,qewi_city,qewi_state,qewi_zip,qewi_nys_lic_no,owner_name,owner_bus_name,owner_bus_street_name,owner_city,owner_zip,owner_state,filing_date,filing_status,prior_cycle_filing_date,prior_status,field_inspection_completed_date,qewi_signed_date,late_filing_amt,failure_to_file_amt,failure_to_collect_amt,comments
0,TR6-716580-7A-N1,716580,Auto-Generated,7,1063335,900,RIVERSIDE DRIVE,MANHATTAN,2136,167,NaN,NaN,No Report Filed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Report Filed,NaN,NaN,NaN,NaN,55000.0,19000.0,0.0,NaN
1,TR6-716657-7C-N1,716657,Auto-Generated,7,1087050,116,EAST 125 STREET,MANHATTAN,1773,7,NaN,NaN,UNSAFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Report Filed,NaN,NaN,NaN,NaN,18000.0,5000.0,20000.0,Civil Penalty Stopped on: 3/25/2019
2,TR6-815020-8C-I1,815020,Initial,8,4610766,2,BAY CLUB DRIVE,QUEENS,5893,7501,2.0,2019-07-30 00:00:00,SWARMP,CHARLES MERRITT,"MERRITT ENGINEERING CONSULTING, P.C",28-08 BAYSIDE LANE,FLUSHING,NY,11358,PE - 048838,MICHAEL BRENNAN,THE BAY CLUB CONDON,1 BAY CLUB DR,BAYSIDE,11360,NY,2019-07-30T00:00:00.000,UNSAFE,NaN,NaN,2019-07-11T00:00:00.000,2019-07-24T20:00:00.000,1500.0,0.0,0.0,DUE TO GENERATING ERROR PREMISE RE-ADDED TO FI...
3,TR6-815020-8C-A1,815020,Amended,8,4610766,2,BAY CLUB DRIVE,QUEENS,5893,7501,2.0,2020-01-23 00:00:00,SWARMP,CHARLES MERRITT,"MERRITT ENGINEERING CONSULTING, P.C",28-08 BAYSIDE LANE,FLUSHING,NY,11358,PE - 048838,MICHAEL BRENNAN,THE BAY CLUB CONDON,1 BAY CLUB DR,BAYSIDE,11360,NY,2020-01-23T00:00:00.000,SWARMP,2019-07-30T00:00:00.000,UNSAFE,2019-12-06T00:00:00.000,2020-01-29T19:00:00.000,1500.0,0.0,0.0,DUE TO GENERATING ERROR PREMISE RE-ADDED TO FI...
4,TR6-815020-8C-N1,815020,Auto-Generated,8,4610766,2,BAY CLUB DRIVE,QUEENS,5893,7501,2.0,NaN,SWARMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Report Filed,NaN,NaN,NaN,NaN,1500.0,0.0,0.0,DUE TO GENERATING ERROR PREMISE RE-ADDED TO FI...


In [7]:
# DataFrame data types
facade_con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   tr6_no                           1000 non-null   object 
 1   control_no                       1000 non-null   int64  
 2   filing_type                      1000 non-null   object 
 3   cycle                            1000 non-null   int64  
 4   bin                              1000 non-null   int64  
 5   house_no                         1000 non-null   object 
 6   street_name                      1000 non-null   object 
 7   borough                          1000 non-null   object 
 8   block                            1000 non-null   int64  
 9   lot                              1000 non-null   int64  
 10  sequence_no                      959 non-null    float64
 11  submitted_on                     957 non-null    object 
 12  current_status       

In [8]:
facade_con.describe(include='all')

,tr6_no,control_no,filing_type,cycle,bin,house_no,street_name,borough,block,lot,sequence_no,submitted_on,current_status,qewi_name,qewi_bus_name,qewi_bus_street_name,qewi_city,qewi_state,qewi_zip,qewi_nys_lic_no,owner_name,owner_bus_name,owner_bus_street_name,owner_city,owner_zip,owner_state,filing_date,filing_status,prior_cycle_filing_date,prior_status,field_inspection_completed_date,qewi_signed_date,late_filing_amt,failure_to_file_amt,failure_to_collect_amt,comments
count,1000,1000.000000,1000,1000.000000,1.000000e+03,1000,1000,1000,1000.000000,1000.000000,959.000000,957,755,958,958,958,958,958,847,847,847,962,962,962,847,847,957,1000,825,824,947,856,994.000000,994.000000,994.000000,180
unique,993,NaN,4,NaN,NaN,603,497,5,NaN,NaN,NaN,286,4,245,256,235,78,4,99,191,546,765,585,79,163,8,282,4,546,3,304,328,NaN,NaN,NaN,153
top,TR6-814993-8A-N1,NaN,Initial,NaN,NaN,1,BROADWAY,MANHATTAN,NaN,NaN,NaN,2020-10-30 00:00:00,UNSAFE,DAVID SALAMON,SALAMON ENGINEERING PLLC,159 WEST 25TH STREET,NEW YORK,NY,10001,PE - 060930,RICHARD MORRISON,PR,24-02 49TH AVENUE,NEW YORK,10016,NY,2020-10-30T00:00:00.000,SWARMP,2013-02-20T00:00:00.000,SAFE,2020-10-28T00:00:00.000,2020-10-27T20:00:00.000,NaN,NaN,NaN,INSPECTION 7/15/15 SUBJECT LETTER SENT 11/3/16
freq,4,NaN,912,NaN,NaN,10,39,646,NaN,NaN,NaN,105,266,33,32,52,498,851,136,32,37,58,24,479,50,822,99,340,22,384,25,22,NaN,NaN,NaN,4
mean,NaN,866273.644000,NaN,8.589000,1.814745e+06,NaN,NaN,NaN,2185.592000,1394.919000,1.196038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12958.299799,4434.607646,2569.416499,NaN
std,NaN,99962.819962,NaN,0.977258,1.187477e+06,NaN,NaN,NaN,2593.734585,2873.378772,0.960137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12164.853665,4078.318066,8829.919741,NaN
min,NaN,600001.000000,NaN,6.000000,1.000005e+06,NaN,NaN,NaN,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
25%,NaN,901718.250000,NaN,9.000000,1.024772e+06,NaN,NaN,NaN,606.250000,16.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2250.000000,0.000000,0.000000,NaN
50%,NaN,905962.000000,NaN,9.000000,1.064218e+06,NaN,NaN,NaN,1337.500000,38.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12000.000000,5000.000000,0.000000,NaN
75%,NaN,911677.000000,NaN,9.000000,2.114658e+06,NaN,NaN,NaN,2457.250000,100.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17250.000000,6000.000000,0.000000,NaN


Are there houses with no adress? - No.

In [9]:
facade_con['house_no'].isna().sum()

0

The possible values 'status' of a building's address

In [10]:
#set(facade_con['house_no'].values.tolist())

We can see that, while there are no missing values, some buildings location are actually a range ("1-10") - probably meaning they are a long *'block'* building. 
This will require attantion when we will try to retrive images from google

### separte 'Safe' and 'Unsafe' building

In [11]:
safe_df = facade_con.loc[(facade_con['filing_status'] == 'SAFE')]
unsafe_df = facade_con.loc[(facade_con['filing_status'] == 'UNSAFE')]

In [12]:
# add  'SWARMP' buildings Safe with a Repair and Maintenance Program (building code classification; New York, NY)
swarmp_df = facade_con.loc[(facade_con['filing_status'] == 'SWARMP')]

In [13]:
# merge SAFE and SWARMP
frames = [safe_df, swarmp_df]
safe_mrg_df = pd.concat(frames, sort=False)

In [14]:
unsafe_df.shape

(292, 36)

In [15]:
safe_df.shape

(324, 36)

In [16]:
swarmp_df.shape

(340, 36)

In [17]:
safe_mrg_df.shape

(664, 36)

*QEWI* means Qualified Exterior Wall Inspector

In [43]:
safe_mrg_df

,tr6_no,control_no,filing_type,cycle,bin,house_no,street_name,borough,block,lot,sequence_no,submitted_on,current_status,qewi_name,qewi_bus_name,qewi_bus_street_name,qewi_city,qewi_state,qewi_zip,qewi_nys_lic_no,owner_name,owner_bus_name,owner_bus_street_name,owner_city,owner_zip,owner_state,filing_date,filing_status,prior_cycle_filing_date,prior_status,field_inspection_completed_date,qewi_signed_date,late_filing_amt,failure_to_file_amt,failure_to_collect_amt,comments,address
14,TR6-815027-8A-I1,815027,Initial,8,1084518,2937,8 AVENUE,MANHATTAN,2106,3,6.0,2019-11-26 00:00:00,SAFE,KENNETH SANTELLI,KENNETH A SANTELLI,480 NORTH BROADWAY,YONKERS,NY,10701,RA - 028770,LLOYD VALDEZ,NYCHA,90 CHURCH STREET,NEW YORK,10007,NY,2019-11-26T00:00:00.000,SAFE,NaN,NaN,2019-10-16T00:00:00.000,NaN,0.0,0.0,0.0,NaN,"2937 8 AVENUE, MANHATTAN, New York, New York"
21,TR6-816622-8C-S1,816622,Subsequent,8,1085588,550,1st Ave,MANHATTAN,962,7501,5.0,2020-01-15 00:00:00,SAFE,HOWARD ZIMMERMAN,HLZIMMERMAN ARCHITECTS,11 W 30 ST,NY,NY,10001,RA - 014327,RICHARD COHEN,NYU LANGONE MEDICAL CENTER/NYU HOSPITALS CENTER,545 1ST AVENUE,NEW YORK,10016,NY,2020-01-15T00:00:00.000,SAFE,2018-07-02T00:00:00.000,SWARMP,2019-12-13T00:00:00.000,2019-12-31T19:00:00.000,0.0,0.0,0.0,NaN,"550 1st Ave, MANHATTAN, New York, New York"
25,TR6-814977-8C-I2,814977,Initial,8,2013306,333,EAST 181 STREET,BRONX,3143,36,2.0,2017-10-02 00:00:00,SAFE,GEORGE XERAKIAS,GEORGE J. XERAKIAS,160-14 12TH AVENUE,WHITESTONE,NY,11357,PE - 088368,PETER LEPORE,"APARTMENT MANAGEMENT ASSOCIATES, LL",2611 WEST 2ND STREET,BROOKLYN,11223,NY,2017-10-02T00:00:00.000,SAFE,2007-03-21T00:00:00.000,SWARMP,2017-10-02T00:00:00.000,2017-10-03T00:00:00.000,0.0,0.0,0.0,NaN,"333 EAST 181 STREET, BRONX, New York, New York"
27,TR6-905332-9A-I1,905332,Initial,9,1045585,1293,YORK AVENUE,MANHATTAN,1464,21,1.0,2020-04-28 00:00:00,SAFE,BRIAN SULLIVAN,SULLIVAN ENGINEERING,409 MINNISINK ROAD,TOTOWA,NJ,07512,PE - 088575,DMITRI KONON,WEILL CORNELL MEDICAL COLLEGE,445 EAST 69TH STREET,NEW YORK,10021,NY,2020-04-28T00:00:00.000,SAFE,2017-05-03T00:00:00.000,SWARMP,2020-03-13T00:00:00.000,2020-05-07T20:00:00.000,0.0,0.0,0.0,NaN,"1293 YORK AVENUE, MANHATTAN, New York, New York"
33,TR6-905428-9C-A1,905428,Amended,9,1046650,995,FIFTH AVENUE,MANHATTAN,1492,69,1.0,2020-07-20 00:00:00,SAFE,STEVE LANE,DEVON ARCHITECTS,20 JERUSALEM AVENUE,HICKSVILLE,NY,11801,RA - 020904,EAMON EARLY,995 5th Avenue. LLC,770 LEXINGTON AVENUE,NEW YORK,10065,NY,2020-07-20T00:00:00.000,SAFE,2020-04-30T00:00:00.000,UNSAFE,2020-07-09T00:00:00.000,2020-08-05T20:00:00.000,17750.0,5000.0,0.0,NaN,"995 FIFTH AVENUE, MANHATTAN, New York, New York"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,TR6-600094-NA-I1,600094,Initial,6,1000833,76,BEAVER STREET,MANHATTAN,28,12,1.0,2007-02-21 00:00:00,SWARMP,MICHEAL V NATARO,ZASKORSKI NOTARO ARCHITECTS LLP,575 8TH AVENUE SUITE 2212,NY,NY,NaN,NaN,NaN,11 HANOVER SQUARE CORP,855 LEXINGTON AVENUE,NY,NaN,NaN,2007-02-21T00:00:00.000,SWARMP,2000-03-01T00:00:00.000,SAFE,2006-12-04T00:00:00.000,2007-02-06T00:00:00.000,0.0,0.0,0.0,NaN,"76 BEAVER STREET, MANHATTAN, New York, New York"
991,TR6-600095-NA-A1,600095,Amended,6,1000835,82,BEAVER STREET,MANHATTAN,28,7501,1.0,2007-11-28 00:00:00,SWARMP,JOHN WHELAN,GOLDSTEIN ASSOCIATES CONSULTING,31 WEST 27TH STREET,NEW YORK,NY,NaN,NaN,NaN,COCOA PARTNERSHIP L.P.,82 BEAVER STREET,NY,NaN,NaN,2007-11-28T00:00:00.000,SWARMP,NaN,NaN,NaN,NaN,150.0,0.0,2000.0,NaN,"82 BEAVER STREET, MANHATTAN, New York, New York"
997,TR6-600101-NA-I1,600101,Initial,6,1000849,61,BROAD STREET,MANHATTAN,29,70,1.0,2007-02-08 00:00:00,SWARMP,HOWARD L ZIMMERMAN,HOWARD ZIMMERMAN ARCHITECTS P.C,11 WEST 30TH STREET,NY,NY,NaN,NaN,NaN,75 BROAD LLC,75 BROAD STREET,N.Y.,NaN,NaN,2007-02-08T00:00:00.000,SWARMP,2002-02-21T00:00:00.000,SWARMP,2007-01-23T00:00:00.000,2007-02-02T00:00:00.000,0.0,0.0,3000.0,NaN,"61 BROAD STREET, MANHATTAN, New Y

## Get the adresses:

For each DF, add an *address* column who's value's a string concatunating the *house_no*, *street_name*, and 'borough' + Ending with "New York, New York"

In [19]:
safe_mrg_df['address'] = safe_mrg_df['house_no'].astype(str) + ' ' + safe_mrg_df['street_name'].astype(str) + ', ' + safe_mrg_df['borough'].astype(str) + ', New York, New York'
unsafe_df['address'] = unsafe_df['house_no'].astype(str) + ' ' + unsafe_df['street_name'].astype(str) + ', ' + unsafe_df['borough'].astype(str) + ', New York, New York'

<ipython-input-19-68e37cf459b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsafe_df['address'] = unsafe_df['house_no'].astype(str) + ' ' + unsafe_df['street_name'].astype(str) + ', ' + unsafe_df['borough'].astype(str) + ', New York, New York'


## Convert physical addresses to Geographic locations → Latitude and Longitude 

Import libraries

In [33]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [34]:
from geopy.extra.rate_limiter import RateLimiter
geocoder = Nominatim(user_agent = 'ironhack') # notice changing the user_agent name in other projects
geocode = RateLimiter(geocoder.geocode, min_delay_seconds=1) # to delay between geocoding calls

In [ ]:
# create location column
safe_mrg_df['location'] = safe_mrg_df['address'].apply(geocode)
unsafe_df['location'] = unsafe_df['address'].apply(geocode)
# create longitude,laatitude and altitude from location column (returns tuple)
unsafe_df['point'] = unsafe_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
safe_mrg_df['point'] = safe_mrg_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

split point column into latitude, longitude and altitude columns

In [134]:
safe_mrg_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(safe_mrg_df['point'].tolist(), index=safe_mrg_df.index)
unsafe_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(unsafe_df['point'].tolist(), index=unsafe_df.index)

C:\Users\meita\anaconda3\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


remove rows that didn't return a location succusfuly

In [136]:
safe_mrg_df = safe_mrg_df[pd.notnull(safe_mrg_df["latitude"])]
unsafe_df = unsafe_df[pd.notnull(unsafe_df["latitude"])]

## Visualize dots

In [172]:
map_save_unsafe = folium.Map(
    location=[40.730610,-73.935242], # (lan, lat) of NYC
    tiles='Stamen Toner',
    zoom_start=12,
)

In [ ]:
safe_mrg_df.apply(lambda row:folium.Circle(location=[row["latitude"], row["longitude"]], color='CornflowerBlue', radius=1.5, popup = row["address"]).add_to(map_save_unsafe), axis=1)
unsafe_df.apply(lambda row:folium.Circle(location=[row["latitude"], row["longitude"]], color='crimson', radius=1.5, popup = row["address"]).add_to(map_save_unsafe), axis=1)

map_save_unsafe # dispky

In [ ]:
unsafe_df

# Scraping google images using Selenium

In [163]:
unsafe_df['location']

KeyError: 'location'

### Import libraries

General

In [26]:
# for printing current time when crashes and waits
import time
import datetime

# for scraping htmls
from bs4 import BeautifulSoup

# to avoid getting blocked
import random

import requests  # for soup

import re # for regex search
import lxml # better then html detection 
import cchardet # to assit in charecter detection

Webdriver specific

In [230]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # for typing
from selenium.webdriver.chrome.options import Options # for loading the page : start only after loading
# for waiting on elemnts to load:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Chrome.options.add_argument("--headless")

# for opening the web browser
PATH = "C:\\Program Files (x86)\\chromedriver.exe" 

In [242]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_experimental_option("detach", True)

## Functions

In [193]:
gmaps_url_base = 'https://www.google.com/maps/place/'

Initialize the webdriver and accept cookies

In [255]:
def open_gmaps(gmaps_url_base):
    driver = webdriver.Chrome(PATH)
    driver.implicitly_wait(10)
    driver.get(gmaps_url_base)

Click on the 'accept cookies' button at start

In [256]:
def accept_cookies():
    # RUN ONLY ONCE
    # allow time for page to load
    #time.sleep(30)
    #WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]')))
    # we can't find the elemnt without switching to its frame first:
    driver.switch_to.frame(driver.find_element_by_xpath("//iframe[contains(@src, 'consent.google.com')]")) 
    time.sleep(random.uniform(2, 8)) # look human
    driver.find_element_by_xpath('//*[@id="introAgreeButton"]/span/span').click()
    #driver.switch_to().parentFrame(); # go back to defualt pane
    print("CLICKED: cockies")

Use search bar

In [270]:
def search_location(address):
    time.sleep(random.uniform(2, 5)) # look human
    driver.find_element_by_xpath('//*[@id="searchbox-searchbutton"]').send_keys(f"{address}",Keys.ENTER)

Try and move to street view

In [354]:
def clk_str_view():
    time.sleep(random.uniform(2, 5)) # look human
    
    # Is there more than 1 result?
    if len(driver.find_elements_by_css_selector('.section-partial-interpretation-title')) > 0: # if there's multiple options
        print("Multiple results found\n")
        driver.find_element_by_css_selector('.section-result-content :first').click() # click the first one
        print("Clicking first\n")
    
    # capture change (if made)
    before_str_view_url = str(driver.current_url)

    # Is there streetview available? 
    tmp_elem = driver.find_elements_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[1]/div[1]/button')
    if len(tmp_elem) > 0:
        tmp_elem[0].click()
        
        if len(driver.find_elements_by_css_selector('#titlecard > div.widget-titlecard.widget-titlecard-show-settings-menu')) > 0:
        #str(driver.current_url) == before_str_view_url:
            print('Streetview exists\n')
            return(True)
        else:
            print('No Streetview\n')
            return(False)

Close side pane

In [300]:
def cls_side_pane():
    driver.find_element_by_xpath('//*[@id="pane"]/div/div[3]/button').click()
    time.sleep(random.uniform(1, 5)) # look human

Hide all besides the image background - **NOTICE** the mark stays 

In [301]:
def visib_hide():    
    # the water mark doesn't remove?
    css_elems = ['.widget-image-header-close',
                '.widget-image-header-scrim',
                '.watermark',
                '.app-viewcard-strip',
                '.scene-footer',
                '#pane',
                '#titlecard']
    
    for elem in css_elems:
        driver.execute_script(f"document.querySelector('{elem}').style.display = 'none';")
        
    time.sleep(random.uniform(1, 3)) # look human

In [357]:
def visib_show():    
    # the water mark doesn't remove?
    css_elems = ['.widget-image-header-close',
                '.widget-image-header-scrim',
                '.watermark',
                '.app-viewcard-strip',
                '.scene-footer',
                '#pane',
                '#titlecard']
    
    for elem in css_elems:
        driver.execute_script(f"document.querySelector('{elem}').style.display = 'unset';")
        
    time.sleep(random.uniform(1, 3)) # look human

In [362]:
def reset_for_search():
    visib_show()
    cls_side_pane() # here it actually 'opens' not as the name suggests
    time.sleep(random.uniform(1, 3)) # look human
    driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[1]/button[1]').click() # click no 'return'

Take screenshot

In [302]:
def scrn_shot(img_num, address):
    driver.save_screenshot(f"./final/str_view/{img_num}_{address}.png") # notice saved location

Pack functions into a larger function

In [370]:
def url_to_image(img_num, address):
    if clk_str_view(): # if ther's a streetview option
        print("Moving to Stretview\n")
        cls_side_pane()
        visib_hide()
        scrn_shot(img_num, address)
        reset_for_search() # go back to main search page > ready for the next search

# WIP

In [365]:
safe_address_lst = safe_mrg_df.address.tolist()

In [366]:
safe_address_lst = safe_address_lst[:3] # just for tests

In [367]:
safe_address_lst

['2937 8 AVENUE, MANHATTAN, New York, New York',
 '550 1st Ave, MANHATTAN, New York, New York',
 '333 EAST 181 STREET, BRONX, New York, New York']

In [307]:
# NOT CURRENTLY WORKING 
try:
    driver = webdriver.Chrome(PATH)
    driver.implicitly_wait(10)
    driver.get(gmaps_url_base)
    #open_gmaps(gmaps_url_base)
    print('opened window')
    accept_cookies()
    print('accepted cookies')
    for img_num, address in enumerate(safe_address_lst):
        print(address, img_num)
        time.sleep(random.uniform(1, 5)) # look human
        #tmp_url = gmaps_url_base + address
        search_location(address)
        time.sleep(random.uniform(3, 10)) # look human
        url_to_image(img_num, address) # for ecach address
except:
    driver.quit()

opened window
CLICKED: cockies
accepted cookies
2937 8 AVENUE, MANHATTAN, New York, New York 0


In [372]:
driver = webdriver.Chrome(PATH)
driver.implicitly_wait(10)
driver.get(gmaps_url_base)
#open_gmaps(gmaps_url_base)
print('opened window')
accept_cookies()
print('accepted cookies')
for img_num, address in enumerate(safe_address_lst):
    print(address, img_num)
    time.sleep(random.uniform(1, 5)) # look human
    #tmp_url = gmaps_url_base + address
    search_location(address)
    time.sleep(random.uniform(3, 10)) # look human
    url_to_image(img_num, address) # for ecach address

opened window
CLICKED: cockies
accepted cookies
2937 8 AVENUE, MANHATTAN, New York, New York 0
https://www.google.com/maps/place/2937+Frederick+Douglass+Blvd,+New+York,+NY+10039,+%C3%89tats-Unis/@40.829973,-73.93976,17z/data=!3m1!4b1!4m5!3m4!1s0x89c2f6804af1e96d:0x95fdcbf6a6599d15!8m2!3d40.829969!4d-73.9375713


https://www.google.com/maps/place/2937+Frederick+Douglass+Blvd,+New+York,+NY+10039,+%C3%89tats-Unis/@40.829973,-73.93976,17z/data=!3m1!4b1!4m5!3m4!1s0x89c2f6804af1e96d:0x95fdcbf6a6599d15!8m2!3d40.829969!4d-73.9375713
No Streetview

550 1st Ave, MANHATTAN, New York, New York 1
Multiple results found



InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=87.0.4280.141)


In [371]:
driver.quit()

In [47]:
print(safe_mrg_df.address.tolist()[3])
geocode(safe_mrg_df.address.tolist()[3])


1293 YORK AVENUE, MANHATTAN, New York, New York


Location(1293, York Avenue, Lenox Hill, Manhattan Community Board 8, Manhattan, New York County, New York, 10021, United States, (40.7651843, -73.9551931, 0.0))